In [4]:
# this data Validation module will check missing data and invalid data
# calculate some statistic number on Missing data 
# calculate simple correlation of the 1 period return
#%%  
import pandas as pd
import numpy as np
import datetime
import scipy.stats as stats
import plotly.graph_objects as go

# read the trading data into a pandas data frame
df = pd.read_csv('quantTest_data.csv',header = None, names = ['date','ts1','ts2'], index_col = 0)

# Convert date col to readarble date time
df['date_time'] = [(datetime.datetime(1,1,1)+ datetime.timedelta(epoch_time)- datetime.timedelta(days=367)).strftime('%Y%m%d %H:%M:%S') for epoch_time in df.index]
df['m_ts1'] = pd.Series()
df['m_ts2'] = pd.Series()
# check for missing data and count the number of missing values in each column
df[['m_ts1', 'm_ts2']]= df[['ts1', 'ts2']].isnull()
df['date_time'] = pd.to_datetime(df['date_time'])
# group data in year and count % missing value of each column 
yearly_data = df.groupby(df['date_time'].dt.year).mean()
# group data in hour and count % missing value of each column
hourly_data = df.groupby(df['date_time'].dt.hour).mean()

#print(yearly_data)
fig = go.Figure()
fig.add_trace(go.Bar(x = yearly_data.index, y =yearly_data['m_ts1'], name='ts1', text=[f"{val:.2%}" for val in yearly_data['m_ts1']],  texttemplate='%{text}',)) #, x= yearly_data.index, y= 'm_ts1', labels={'x':'Year', 'y':'% of Missing Data'}))
fig.add_trace(go.Bar(x = yearly_data.index, y =yearly_data['m_ts2'], name='ts2', text=[f"{val:.2%}" for val in yearly_data['m_ts2']],  texttemplate='%{text}',)) 

# Set the layout of the figure
fig.update_layout(
    title='Histograms for % of missing data of ts1 and ts2 by YEAR',
    xaxis_title='Year',
    yaxis_title='% Missing Data'
)
# Show the plot
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Bar(x = hourly_data.index, y =hourly_data['m_ts1'], name='ts1', text=[f"{val:.2%}" for val in hourly_data['m_ts1']],  texttemplate='%{text}',)) #, x= yearly_data.index, y= 'm_ts1', labels={'x':'Year', 'y':'% of Missing Data'}))
fig2.add_trace(go.Bar(x = hourly_data.index, y =hourly_data['m_ts2'], name='ts2', text=[f"{val:.2%}" for val in hourly_data['m_ts2']],  texttemplate='%{text}',)) 

# Set the layout of the figure
fig2.update_layout(
    title='Histograms for % of missing data of ts1 and ts2 by Hour of the Day',
    xaxis=dict(
        title = 'Hour of the Day',
        automargin=False,
        tickangle=45,
        dtick = 1
        ),
    yaxis_title='% Missing Data'
)
# Show the plot
fig2.show()
# print(yearly_data, hourly_data)


df['r_ts1'] = np.log(df['ts1']/df['ts1'].shift(1))
df['r_ts2'] = np.log(df['ts2']/df['ts2'].shift(1))

df = df.dropna()
# calculate the Pearson correlation coefficient and the p-value
tcorr, p_value = stats.pearsonr(df['r_ts1'], df['r_ts2'])
print(tcorr, p_value)

# %%


-0.16241017030791316 0.0


In [5]:
# this module test 
# 1. correlation of i periods' return between ts1 and the same period return for ts2
# 2. correlation for ts1 past return vs ts2 future return
# 3. correlation for ts2 past return vs ts1 future return
# 4. correlation for ts1 past return vs ts1 future return
# 5. correlation for ts2 past return vs ts2 future return
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import scipy.stats as stats

# read the trading data into a pandas data frame
df = pd.read_csv('quantTest_data.csv',header = None, names = ['date','ts1','ts2'], index_col = 0)

# correlation for ts1 vs ts2 return look back i period
corr1 = pd.DataFrame()
id = 1
for i in range(200, 10000, 200):
    r_df = pd.DataFrame(index = df.index)
    # return of ts1 & ts2 for i periods
    r_df['ts1_p'] = np.log(df['ts1']/df['ts1'].shift(i))     
    r_df['ts2_p'] = np.log(df['ts2']/df['ts2'].shift(i))
    r_df = r_df.dropna()
    # calculate the Pearson correlation coefficient and the p-value
    tcorr, p_value = stats.pearsonr(r_df['ts1_p'], r_df['ts2_p'])
    # store correlation value
    corr1.loc[id,'lookback_period'] = i
    corr1.loc[id, 'corr'] = tcorr
    corr1.loc[id, 'p_value'] = p_value
    # number of valid data point
    corr1.loc[id, 'rows'] = r_df.shape[0]
    id+= 1
corr1.to_csv('ts1ts2corr.csv')   

fig = px.scatter(corr1, x='lookback_period', y='corr', title='Correlation between ts1 vs ts2 based on X period rolling return', )
fig.update_xaxes(title='Return look back periods')
fig.show()

# correlation for ts1 past return vs ts2 future return
corr2 = pd.DataFrame()
id = 1
# i = period for predicted return
# j = period for look back return 
for i in range(500, 10000, 500):
    for j in range(500, 10000, 500):
        r_df = pd.DataFrame()
        # return of ts1 for j periods starting at i lookback period
        r_df['lookback_p'] = np.log(df['ts1'].shift(i)/df['ts1'].shift(i+j))   
        # return of ts1 for i periods 
        r_df['predict_p'] = np.log(df['ts2']/df['ts2'].shift(i))    
        r_df = r_df.dropna()
        # calculate the Pearson correlation coefficient and the p-value
        tcorr, p_value = stats.pearsonr(r_df['predict_p'], r_df['lookback_p'])
        #print(i, r_df.shape[0], tcorr, p_value)
        corr2.loc[id,'predict_period'] = i
        corr2.loc[id,'lookback_period'] = j
        corr2.loc[id, 'corr'] = tcorr
        corr2.loc[id, 'p_value'] = p_value
        corr2.loc[id, 'rows'] = r_df.shape[0]
        id+= 1
corr2.to_csv('ts1pre_corr.csv')   

fig1 = px.scatter_3d(corr2, x='predict_period', y='lookback_period', z='corr', color='lookback_period')
fig1.show()

# correlation for ts2 past return vs ts1 future return
corr3 = pd.DataFrame()
id = 1
for i in range(500, 10000, 500):
    for j in range(500, 10000, 500):
        r_df = pd.DataFrame()
        # return of ts1 for j periods starting at i lookback period
        r_df['lookback_p'] = np.log(df['ts2'].shift(i)/df['ts2'].shift(i+j))   
        # return of ts1 for i periods 
        r_df['predict_p'] = np.log(df['ts1']/df['ts1'].shift(i))    
        r_df = r_df.dropna()
        # calculate the Pearson correlation coefficient and the p-value
        tcorr, p_value = stats.pearsonr(r_df['predict_p'], r_df['lookback_p'])
        #print(i, r_df.shape[0], tcorr, p_value)
        corr3.loc[id,'predict_period'] = i
        corr3.loc[id,'lookback_period'] = j
        corr3.loc[id, 'corr'] = tcorr
        corr3.loc[id, 'p_value'] = p_value
        corr3.loc[id, 'rows'] = r_df.shape[0]
        id+= 1
corr3.to_csv('ts2pre_corr.csv')   

fig2 = px.scatter_3d(corr3, x='predict_period', y='lookback_period', z='corr', color='lookback_period')
fig2.show()


# correlation for ts1 past return vs ts1 future return
corr4 = pd.DataFrame()
id = 1
# i = period for predicted return
# j = period for look back return 
for i in range(500, 10000, 500):
    for j in range(500, 10000, 500):
        r_df = pd.DataFrame()
        # return of ts1 for j periods starting at i lookback period
        r_df['lookback_p'] = np.log(df['ts1'].shift(i)/df['ts1'].shift(i+j))   
        # return of ts1 for i periods 
        r_df['predict_p'] = np.log(df['ts1']/df['ts1'].shift(i))    
        r_df = r_df.dropna()
        # calculate the Pearson correlation coefficient and the p-value
        tcorr, p_value = stats.pearsonr(r_df['predict_p'], r_df['lookback_p'])
        #print(i, r_df.shape[0], tcorr, p_value)
        corr4.loc[id,'predict_period'] = i
        corr4.loc[id,'lookback_period'] = j
        corr4.loc[id, 'corr'] = tcorr
        corr4.loc[id, 'p_value'] = p_value
        corr4.loc[id, 'rows'] = r_df.shape[0]
        id+= 1
corr4.to_csv('ts1_corr.csv')   

fig3 = px.scatter_3d(corr4, x='predict_period', y='lookback_period', z='corr', color='lookback_period')
fig3.show()

# correlation for ts2 past return vs ts2 future return
corr5 = pd.DataFrame()
id = 1
for i in range(500, 10000, 500):
    for j in range(500, 10000, 500):
        r_df = pd.DataFrame()
        # return of ts1 for j periods starting at i lookback period
        r_df['lookback_p'] = np.log(df['ts2'].shift(i)/df['ts2'].shift(i+j))   
        # return of ts1 for i periods 
        r_df['predict_p'] = np.log(df['ts2']/df['ts2'].shift(i))    
        r_df = r_df.dropna()
        # calculate the Pearson correlation coefficient and the p-value
        tcorr, p_value = stats.pearsonr(r_df['predict_p'], r_df['lookback_p'])
        #print(i, r_df.shape[0], tcorr, p_value)
        corr5.loc[id,'predict_period'] = i
        corr5.loc[id,'lookback_period'] = j
        corr5.loc[id, 'corr'] = tcorr
        corr5.loc[id, 'p_value'] = p_value
        corr5.loc[id, 'rows'] = r_df.shape[0]
        id+= 1
corr5.to_csv('ts2_corr.csv')   

fig4 = px.scatter_3d(corr5, x='predict_period', y='lookback_period', z='corr', color='lookback_period')
fig4.show()

In [7]:
# this module test the relationship between top percentile of largest absolute past return vs future return
# 1. correlation for top 20% largest ts1 absolute past return vs ts2 future return
# 2. correlation for top 30% largest ts1 absolute past return vs ts2 future return
# 3. correlation for top 50% largest ts1 absolute past return vs ts2 future return
#%%  
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import scipy.stats as stats

# read the trading data into a pandas data frame
df = pd.read_csv('quantTest_data.csv',header = None, names = ['date','ts1','ts2'], index_col = 0)
corr = pd.DataFrame()
id = 1
# top n percentile of largest absolute past return/trend
n1 = 20
n2 = 30 
n3 = 50
#50 #10
# i = predicted return period,  j = lookback period 
# correlation for ts1 past return vs ts2 future return
for i in range(400, 9000, 400):
    for j in range(400, 9000, 400):
        r_df = pd.DataFrame(index = df.index)
        r_df['lookback_p'] = np.log(df['ts1'].shift(i)/df['ts1'].shift(i+j)) 
        r_df['predict_p'] = np.log(df['ts2']/df['ts2'].shift(i))             
        r_df = r_df.dropna()
        # calculate abs return
        r_df['lookback_abs'] = np.abs(r_df['lookback_p'])
        # Calculate n percentile of absolute rolling_ts2 values
        percentile_n1 = np.percentile(r_df['lookback_abs'], 100-n1)
        percentile_n2 = np.percentile(r_df['lookback_abs'], 100-n2)
        percentile_n3 = np.percentile(r_df['lookback_abs'], 100-n3)        
        # Filter absolute rolling_ts2 values based on 95th percentile
        filtered1 = r_df[r_df['lookback_abs'] >= percentile_n1]
        filtered2 = r_df[r_df['lookback_abs'] >= percentile_n2]
        filtered3 = r_df[r_df['lookback_abs'] >= percentile_n3]        
        # calculate the Pearson correlation coefficient and the p-value
        tcorr1, p_value1 = stats.pearsonr(filtered1['predict_p'], filtered1['lookback_p'])     
        tcorr2, p_value2 = stats.pearsonr(filtered2['predict_p'], filtered2['lookback_p'])
        tcorr3, p_value3 = stats.pearsonr(filtered3['predict_p'], filtered3['lookback_p'])
        #corr_rolling = filtered['predict_p'].corr(filtered['lookback_p'])
        corr.loc[id,'predict_period'] = i
        corr.loc[id,'lookback_period'] = j
        corr.loc[id, 'corr1'] = tcorr1
        corr.loc[id, 'p_value1'] = p_value1
        corr.loc[id, 'rows1'] = filtered1.shape[0]       
        corr.loc[id, 'corr2'] = tcorr2
        corr.loc[id, 'p_value2'] = p_value2
        corr.loc[id, 'rows2'] = filtered1.shape[0] 
        corr.loc[id, 'corr3'] = tcorr3
        corr.loc[id, 'p_value3'] = p_value3
        corr.loc[id, 'rows3'] = filtered1.shape[0] 
        id+= 1
corr.to_csv('percentile_corr.csv')   
fig = px.scatter_3d(corr, x='predict_period', y='lookback_period', z='corr1', color='lookback_period')
fig.show()

fig2 = px.scatter_3d(corr, x='predict_period', y='lookback_period', z='corr2', color='lookback_period')
fig2.show()

fig3 = px.scatter_3d(corr, x='predict_period', y='lookback_period', z='corr3', color='lookback_period')
fig3.show()